In [1]:
import sys, os
current_working_directory = os.getcwd()
sys.path.append(os.path.dirname(current_working_directory))

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.base_language import BaseLanguageModel
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.summarize import map_reduce_prompt, refine_prompts, stuff_prompt, _load_map_reduce_chain, _load_refine_chain, _load_stuff_chain, LoadingCallable
from langchain.prompts.base import BasePromptTemplate
from typing import Optional, Any, Mapping
from prompts import stuff_prompt, refine_prompt, mapreduce_prompt
from loader.pdf_loader import loadDocsFromPDF
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import milvusUtil as milvus
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from chains.qa_refine_prompt import DEFAULT_REFINE_PROMPT, DEFAULT_TEXT_QA_PROMPT, EXAMPLE_PROMPT
from chains.condense_quest_prompt import CONDENSE_QUESTION_PROMPT_ZH
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.docstore.document import Document

Exception ignored in: <function Client.__del__ at 0x7f789be29940>
Traceback (most recent call last):
  File "/apps/micromamba/envs/chatdoc/lib/python3.9/site-packages/httpx/_client.py", line 1139, in __del__
    self.close()
  File "/apps/micromamba/envs/chatdoc/lib/python3.9/site-packages/httpx/_client.py", line 1111, in close
    self._transport.close()
AttributeError: 'Client' object has no attribute '_transport'


In [2]:
from models.glm_llm import glm_llm as glm
llm = glm()
embedding = HuggingFaceEmbeddings(model_name="/apps/models/GanymedeNil/text2vec-base-chinese/GanymedeNil_text2vec-base-chinese", )


No sentence-transformers model found with name /apps/models/GanymedeNil/text2vec-base-chinese/GanymedeNil_text2vec-base-chinese. Creating a new one with MEAN pooling.


In [5]:
collection_name_zh, collection_name = milvus.get_kb_collection_name(embedding=embedding, query="打开excel知识库")
vectorstore = milvus.get_vectordb(embedding=embedding, collection_name=collection_name,connection_args = None)
chat_history = []

question=打开excel知识库, get_kb_collection_name return page_content='excel测试' metadata={'collection_name': 'k7635e3ebfe08'}, score = 362.1020202636719


In [8]:
chain = load_qa_with_sources_chain(llm, chain_type="refine", question_prompt = DEFAULT_TEXT_QA_PROMPT, refine_prompt = DEFAULT_REFINE_PROMPT, document_prompt = EXAMPLE_PROMPT)
query = "请总结广铁集团机关一般管理人员管理办法？"
qa = RetrievalQAWithSourcesChain(combine_documents_chain=chain, retriever=vectorstore.as_retriever(), return_source_documents = True)
result = qa({"question": query}, return_only_outputs=True)
docs:list[Document] = result["source_documents"]
print(f"【AI的回答】\n{result['answer']}")
for doc in docs:
    print(f"出处：{doc.metadata['file_path']}, page:{doc.metadata['page_number']}")

chat_history.append((query, result["answer"]))

【AI的回答】
根据新的上下文信息，广铁集团机关一般管理人员管理办法适用于集团公司及所属非法人单位(含运输站段、运输辅助单位，以及其他不具有法人资格的单位)、一级非运输企业、集团公司控股合资公司。该办法试行统一管理、分级负责、分类评价、全面监督的原则。该办法的主要内容包括分类考核、考核方式、考核内容以及结果运用。集团公司根据年度综合考评情况，将考评结果纳入集团公司年度经营业绩考核。

新的上下文信息还提供了一些新的细节，例如，广铁集团机关一般管理人员管理办法中规定，机关一般管理人员应当具备政治素质好，坚决贯彻党的路线、方针、政策，坚决执行上级的决策部署，顾全大局，令行禁止；忠于职守、勤勉敬业，主动承责、积极担责，始终保持良好的精神状态；品行端正，遵纪守法，恪守职业道德；公道正派、清正廉洁。此外，新的上下文信息还提供了一些新的示例，例如，如何落实机关一般管理人员管理工作的民主集中制原则。

综上所述，新的上下文信息提供了更多的细节和背景，有助于更好地理解广铁集团机关一般管理人员管理办法的内容和目的。
出处：/apps/docs/excel测试/（广铁企发〔2020〕79号）中国铁路广州局集团有限公司关于公布《广州局集团公司法律事务监督检查考评办法》的发文说明.pdf, page:1
出处：/apps/docs/excel测试/（人函〔2019〕10号）关于公布《中国铁路广州局集团有限公司机关一般管理人员管理办法（试行）》的通知.pdf, page:2
出处：/apps/docs/excel测试/（人函〔2019〕10号）《中国铁路广州局集团有限公司机关一般管理人员管理办法（试行）》.pdf, page:2
出处：/apps/docs/excel测试/（集团办计发〔2019〕67号）中国铁路广州局集团有限公司办公室关于公布《广州局集团公司节约能源管理办法》的通知.pdf, page:9


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory)
vectordbkwargs = {"search_distance": 0.9}
chat_history = []
result = qa({"question": "请总结广铁集团机关一般管理人员管理办法？"})
# result = qa({"question": question,"chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
print(result)

In [ ]:
# from langchain.prompts.prompt import PromptTemplate

# _template = """给定以下对话和后续问题，请将后续问题重新表述为一个独立的问题，使用中文回答.

# 对话历史:
# {chat_history}
# 接下来的输入: {question}
# 独立问题:"""
# CONDENSE_QUESTION_PROMPT_ZH = PromptTemplate.from_template(_template)

# prompt_template = """使用下面的上下文来回答最后的问题。如果你不知道答案，只需要说你不知道，不要试图编造一个答案.

# {context}

# 问题: {question}
# 有帮助的回答:"""
# QA_PROMPT_ZH = PromptTemplate(
#     template=prompt_template, input_variables=["context", "question"]
# )


In [9]:
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT_ZH)
doc_chain = load_qa_with_sources_chain(llm, chain_type="refine", question_prompt = DEFAULT_TEXT_QA_PROMPT, refine_prompt = DEFAULT_REFINE_PROMPT, document_prompt = EXAMPLE_PROMPT)
chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents = True
)
quesyion = "请说明广铁集团机关一般管理人员管理办法什么时候试行"
result = chain({"question": quesyion, "chat_history": chat_history})
docs:list[Document] = result["source_documents"]
source_documents = [{}]
for doc in docs:
    source_documents.append({'file_path':doc.metadata['file_path'], 'page_number':doc.metadata['page_number']})
answer = {"question":result["question"], "answer":result['answer'], "source_documents":source_documents}
chat_history.append((answer["question"], answer["answer"]))
print(f"chat_history.........\n{chat_history}")
print(f"question.........\n{quesyion}")
print(f"answer............\n{answer}")

[('请总结广铁集团机关一般管理人员管理办法？', '根据新的上下文信息，广铁集团机关一般管理人员管理办法适用于集团公司及所属非法人单位(含运输站段、运输辅助单位，以及其他不具有法人资格的单位)、一级非运输企业、集团公司控股合资公司。该办法试行统一管理、分级负责、分类评价、全面监督的原则。该办法的主要内容包括分类考核、考核方式、考核内容以及结果运用。集团公司根据年度综合考评情况，将考评结果纳入集团公司年度经营业绩考核。\n\n新的上下文信息还提供了一些新的细节，例如，广铁集团机关一般管理人员管理办法中规定，机关一般管理人员应当具备政治素质好，坚决贯彻党的路线、方针、政策，坚决执行上级的决策部署，顾全大局，令行禁止；忠于职守、勤勉敬业，主动承责、积极担责，始终保持良好的精神状态；品行端正，遵纪守法，恪守职业道德；公道正派、清正廉洁。此外，新的上下文信息还提供了一些新的示例，例如，如何落实机关一般管理人员管理工作的民主集中制原则。\n\n综上所述，新的上下文信息提供了更多的细节和背景，有助于更好地理解广铁集团机关一般管理人员管理办法的内容和目的。'), ('请说明广铁集团机关一般管理人员管理办法什么时候试行', '根据新的上下文信息，广铁集团机关一般管理人员管理办法是在 2019 年试行的。\n\n新的上下文信息包括：\n\n- 中国铁路广州局集团有限公司人事部人函〔2019〕10 号发布的通知\n- 中国铁路广州局集团有限公司机关一般管理人员管理办法(试行)\n- 第一章总则\n- 第一条为认真贯彻执行党的干部路线方针政策，落实全面从严治党、从严管理干部的要求，适应公司制改革发展需要，建立科学规范、有效管用、有利于优秀人才脱颖而出的选人用人机制，建设一支信念坚定、为民服务、勤政务实、敢于担当、清正廉洁的高素质机关管理人员队伍，依据集团公司人事管理相关制度和有关法律法规，结合实际，制定本办法。\n- 第二条本办法所称机关一般管理人员，包括集团公司机关职能机构、党群机构、附属机构、生产机构正科职及以下管理人员和专业技术人员。\n- 第三条机关一般管理人员管理工作要遵循以下原则：\n- (一)党管干部；\n- (二)德才兼备、以德为先，五湖四海、任人唯贤；\n- (三)事业为上、人岗相适、人事相宜；\n- (四)公道正派、注重实绩、群众公认；\n- (五)民主集中制；\n-

In [10]:
question = "请说明广铁集团机关一般管理人员管理办法的主要内容"
result = chain({"question": question, "chat_history": chat_history})
docs:list[Document] = result["source_documents"]
source_documents = [{}]
for doc in docs:
    source_documents.append({'file_path':doc.metadata['file_path'], 'page_number':doc.metadata['page_number']})
answer = {"question":result["question"], "answer":result['answer'], "source_documents":source_documents}
chat_history.append((answer["question"], answer["answer"]))
print(f"chat_history............\n:{chat_history}")
print(f"question............\n:{question}")
print(f"answer..............\n{answer}")

chat_history............
:[('请总结广铁集团机关一般管理人员管理办法？', '根据新的上下文信息，广铁集团机关一般管理人员管理办法适用于集团公司及所属非法人单位(含运输站段、运输辅助单位，以及其他不具有法人资格的单位)、一级非运输企业、集团公司控股合资公司。该办法试行统一管理、分级负责、分类评价、全面监督的原则。该办法的主要内容包括分类考核、考核方式、考核内容以及结果运用。集团公司根据年度综合考评情况，将考评结果纳入集团公司年度经营业绩考核。\n\n新的上下文信息还提供了一些新的细节，例如，广铁集团机关一般管理人员管理办法中规定，机关一般管理人员应当具备政治素质好，坚决贯彻党的路线、方针、政策，坚决执行上级的决策部署，顾全大局，令行禁止；忠于职守、勤勉敬业，主动承责、积极担责，始终保持良好的精神状态；品行端正，遵纪守法，恪守职业道德；公道正派、清正廉洁。此外，新的上下文信息还提供了一些新的示例，例如，如何落实机关一般管理人员管理工作的民主集中制原则。\n\n综上所述，新的上下文信息提供了更多的细节和背景，有助于更好地理解广铁集团机关一般管理人员管理办法的内容和目的。'), ('请说明广铁集团机关一般管理人员管理办法什么时候试行', '根据新的上下文信息，广铁集团机关一般管理人员管理办法是在 2019 年试行的。\n\n新的上下文信息包括：\n\n- 中国铁路广州局集团有限公司人事部人函〔2019〕10 号发布的通知\n- 中国铁路广州局集团有限公司机关一般管理人员管理办法(试行)\n- 第一章总则\n- 第一条为认真贯彻执行党的干部路线方针政策，落实全面从严治党、从严管理干部的要求，适应公司制改革发展需要，建立科学规范、有效管用、有利于优秀人才脱颖而出的选人用人机制，建设一支信念坚定、为民服务、勤政务实、敢于担当、清正廉洁的高素质机关管理人员队伍，依据集团公司人事管理相关制度和有关法律法规，结合实际，制定本办法。\n- 第二条本办法所称机关一般管理人员，包括集团公司机关职能机构、党群机构、附属机构、生产机构正科职及以下管理人员和专业技术人员。\n- 第三条机关一般管理人员管理工作要遵循以下原则：\n- (一)党管干部；\n- (二)德才兼备、以德为先，五湖四海、任人唯贤；\n- (三)事业为上、人岗相适、人事相宜；\n- (四)公道正派、

In [11]:
question = "广铁集团机关一般管理人员的工作职责是什么"
result = chain({"question": question, "chat_history": chat_history})
docs:list[Document] = result["source_documents"]
source_documents = [{}]
for doc in docs:
    source_documents.append({'file_path':doc.metadata['file_path'], 'page_number':doc.metadata['page_number']})
answer = {"question":result["question"], "answer":result['answer'], "source_documents":source_documents}
chat_history.append((answer["question"], answer["answer"]))
print(f"chat_history............\n{chat_history}")
print(f"question............\n{question}")
print(f"answer..............\n{answer}")

chat_history............
[('请总结广铁集团机关一般管理人员管理办法？', '根据新的上下文信息，广铁集团机关一般管理人员管理办法适用于集团公司及所属非法人单位(含运输站段、运输辅助单位，以及其他不具有法人资格的单位)、一级非运输企业、集团公司控股合资公司。该办法试行统一管理、分级负责、分类评价、全面监督的原则。该办法的主要内容包括分类考核、考核方式、考核内容以及结果运用。集团公司根据年度综合考评情况，将考评结果纳入集团公司年度经营业绩考核。\n\n新的上下文信息还提供了一些新的细节，例如，广铁集团机关一般管理人员管理办法中规定，机关一般管理人员应当具备政治素质好，坚决贯彻党的路线、方针、政策，坚决执行上级的决策部署，顾全大局，令行禁止；忠于职守、勤勉敬业，主动承责、积极担责，始终保持良好的精神状态；品行端正，遵纪守法，恪守职业道德；公道正派、清正廉洁。此外，新的上下文信息还提供了一些新的示例，例如，如何落实机关一般管理人员管理工作的民主集中制原则。\n\n综上所述，新的上下文信息提供了更多的细节和背景，有助于更好地理解广铁集团机关一般管理人员管理办法的内容和目的。'), ('请说明广铁集团机关一般管理人员管理办法什么时候试行', '根据新的上下文信息，广铁集团机关一般管理人员管理办法是在 2019 年试行的。\n\n新的上下文信息包括：\n\n- 中国铁路广州局集团有限公司人事部人函〔2019〕10 号发布的通知\n- 中国铁路广州局集团有限公司机关一般管理人员管理办法(试行)\n- 第一章总则\n- 第一条为认真贯彻执行党的干部路线方针政策，落实全面从严治党、从严管理干部的要求，适应公司制改革发展需要，建立科学规范、有效管用、有利于优秀人才脱颖而出的选人用人机制，建设一支信念坚定、为民服务、勤政务实、敢于担当、清正廉洁的高素质机关管理人员队伍，依据集团公司人事管理相关制度和有关法律法规，结合实际，制定本办法。\n- 第二条本办法所称机关一般管理人员，包括集团公司机关职能机构、党群机构、附属机构、生产机构正科职及以下管理人员和专业技术人员。\n- 第三条机关一般管理人员管理工作要遵循以下原则：\n- (一)党管干部；\n- (二)德才兼备、以德为先，五湖四海、任人唯贤；\n- (三)事业为上、人岗相适、人事相宜；\n- (四)公道正派、注

In [12]:
question = "广铁集团机关一般管理人员的任职条件是什么"
result = chain({"question": question, "chat_history": chat_history})
docs:list[Document] = result["source_documents"]
source_documents = [{}]
for doc in docs:
    source_documents.append({'file_path':doc.metadata['file_path'], 'page_number':doc.metadata['page_number']})
answer = {"question":result["question"], "answer":result['answer'], "source_documents":source_documents}
chat_history.append((answer["question"], answer["answer"]))
print(f"chat_history............\n{chat_history}")
print(f"question............\n{question}")
print(f"answer..............\n{answer}")

chat_history............
[('请总结广铁集团机关一般管理人员管理办法？', '根据新的上下文信息，广铁集团机关一般管理人员管理办法适用于集团公司及所属非法人单位(含运输站段、运输辅助单位，以及其他不具有法人资格的单位)、一级非运输企业、集团公司控股合资公司。该办法试行统一管理、分级负责、分类评价、全面监督的原则。该办法的主要内容包括分类考核、考核方式、考核内容以及结果运用。集团公司根据年度综合考评情况，将考评结果纳入集团公司年度经营业绩考核。\n\n新的上下文信息还提供了一些新的细节，例如，广铁集团机关一般管理人员管理办法中规定，机关一般管理人员应当具备政治素质好，坚决贯彻党的路线、方针、政策，坚决执行上级的决策部署，顾全大局，令行禁止；忠于职守、勤勉敬业，主动承责、积极担责，始终保持良好的精神状态；品行端正，遵纪守法，恪守职业道德；公道正派、清正廉洁。此外，新的上下文信息还提供了一些新的示例，例如，如何落实机关一般管理人员管理工作的民主集中制原则。\n\n综上所述，新的上下文信息提供了更多的细节和背景，有助于更好地理解广铁集团机关一般管理人员管理办法的内容和目的。'), ('请说明广铁集团机关一般管理人员管理办法什么时候试行', '根据新的上下文信息，广铁集团机关一般管理人员管理办法是在 2019 年试行的。\n\n新的上下文信息包括：\n\n- 中国铁路广州局集团有限公司人事部人函〔2019〕10 号发布的通知\n- 中国铁路广州局集团有限公司机关一般管理人员管理办法(试行)\n- 第一章总则\n- 第一条为认真贯彻执行党的干部路线方针政策，落实全面从严治党、从严管理干部的要求，适应公司制改革发展需要，建立科学规范、有效管用、有利于优秀人才脱颖而出的选人用人机制，建设一支信念坚定、为民服务、勤政务实、敢于担当、清正廉洁的高素质机关管理人员队伍，依据集团公司人事管理相关制度和有关法律法规，结合实际，制定本办法。\n- 第二条本办法所称机关一般管理人员，包括集团公司机关职能机构、党群机构、附属机构、生产机构正科职及以下管理人员和专业技术人员。\n- 第三条机关一般管理人员管理工作要遵循以下原则：\n- (一)党管干部；\n- (二)德才兼备、以德为先，五湖四海、任人唯贤；\n- (三)事业为上、人岗相适、人事相宜；\n- (四)公道正派、注